In [1]:
#!pip install timm

In [2]:
import pandas as pd
import timm
import numpy as np
import json
import gc
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
import pytorch_lightning as pl
import albumentations
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from albumentations.core.composition import Compose
from albumentations.pytorch import ToTensorV2
import torchvision
from typing import Any, Dict, List, Union, Optional
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import StratifiedKFold
import cv2
import torch
#from efficientnet_pytorch import EfficientNet
import numpy as np

from PIL import Image
from PIL import ImageFile

In [3]:
base_dir = r'C:\Users\Kaggle\Leaf_Classification\input'

In [4]:
### read csv and label mapping

labels = pd.read_csv(os.path.join(base_dir,'train.csv'))

with open(f'{base_dir}/label_num_to_disease_map.json') as f:
    label_mapper = json.load(f)

### Dataset class

In [5]:
class ImageClassificationDataset(Dataset):
    def __init__(
        self,
        image_names: List,
        transforms: Compose,
        labels: Optional[List[int]],
        img_path: str = '',
        mode: str = 'train',
        labels_to_ohe: bool = False,
        n_classes: int = 5,
    ):
        """
        Image classification dataset.

        Args:
            df: dataframe with image id and bboxes
            mode: train/val/test
            img_path: path to images
            transforms: albumentations
        """

        self.mode = mode
        self.transforms = transforms
        self.img_path = img_path
        self.image_names = image_names
        if labels is not None:
            if not labels_to_ohe:
                self.labels = np.array(labels)
            else:
                self.labels = np.zeros((len(labels), n_classes))
                self.labels[np.arange(len(labels)), np.array(labels)] = 1

    def __getitem__(self, idx: int) -> Dict[str, np.array]:
        image_path = self.img_path + self.image_names[idx]
        image = cv2.imread(f'{image_path}', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if image is None:
            raise FileNotFoundError(image_path)
        target = self.labels[idx]

        img = self.transforms(image=image)['image']
        sample = {'image_path': image_path, 'image': img, 'target': np.array(target).astype('int64')}

        return sample

    def __len__(self) -> int:
        return len(self.image_names)

### Augs

In [6]:
sz = 576
sz_efffnetb4 = 600

In [7]:
TTA0 = albumentations.Compose([
            albumentations.Resize(sz, sz,p=1),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA1 = albumentations.Compose([
            albumentations.RandomResizedCrop(sz, sz),
            albumentations.HorizontalFlip(p=1),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA2 = albumentations.Compose([
            albumentations.RandomResizedCrop(sz, sz),
            albumentations.VerticalFlip(p=1),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)

TTA2_5 = albumentations.Compose([
            albumentations.RandomResizedCrop(sz, sz),
            albumentations.Transpose(p=1.),
           albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA3 = albumentations.Compose([
            albumentations.RandomResizedCrop(sz, sz),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=1
            ),
           albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA4 = albumentations.Compose([
            albumentations.RandomResizedCrop(sz, sz),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=1.
            ),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA5 = albumentations.Compose([
            albumentations.RandomResizedCrop(sz, sz),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            albumentations.Cutout(p=1.), 
            ToTensorV2()],
            p=1.)

TTA6 = albumentations.Compose([
            albumentations.RandomResizedCrop(sz, sz),
            albumentations.ShiftScaleRotate(p=1.),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)

TTA7 = albumentations.Compose([
            albumentations.RandomResizedCrop(sz, sz),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            albumentations.CoarseDropout(p=1.),
            ToTensorV2()],
            p=1.)

In [8]:
infer_augs_effnet = albumentations.Compose([
            albumentations.RandomResizedCrop(sz_efffnetb4, sz_efffnetb4),
            albumentations.Transpose(p=0.5),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.ShiftScaleRotate(p=0.5),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            albumentations.CoarseDropout(p=0.5),
            albumentations.Cutout(p=0.5), 
            ToTensorV2()],
            p=1.)


infer_augs_resnext = albumentations.Compose([
            albumentations.RandomResizedCrop(sz, sz),
            albumentations.Transpose(p=0.5),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.ShiftScaleRotate(p=0.5),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            albumentations.CoarseDropout(p=0.5),
            albumentations.Cutout(p=0.5), 
            ToTensorV2()],
            p=1.)

### PL module

In [9]:
def freeze_until(net: Any, param_name: str = None) -> None:
    """
    Freeze net until param_name


    Args:
        net:
        param_name:

    """
    found_name = False
    for name, params in net.named_parameters():
        if name == param_name:
            found_name = True
        params.requires_grad = found_name

class BasicEncoder(nn.Module):
    def __init__(
        self,
        arch: str = 'resnet18',
        source: str = 'torchvision',
        pretrained: str = None,
        n_layers: int = -1,
        freeze: bool = False,
        to_one_channel: bool = False,
        freeze_until_layer: str = None,
    ) -> None:
        """
        Initialize Encoder.

        Args:
            num_classes: the number of target classes, the size of the last layer's output
            arch: the name of the architecture form pretrainedmodels
            pretrained: the mode for pretrained model from pretrainedmodels
            n_layers: number of layers to keep
            freeze: to freeze model
            freeze_until: freeze until this layer. If None, then freeze all layers
        """
        super().__init__()
        if source == 'timm':
            net = timm.create_model(arch, pretrained=pretrained)
            self.output_dimension = list(net.children())[-1].in_features
        else:
            print("Not implemented")
            
        if freeze:
            freeze_until(net, freeze_until_layer)
        
        layers = list(net.children())[:n_layers]    
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        output = self.layers(x)

        return output

In [10]:
class BasicDecoder(nn.Module):
    def __init__(self, pool_output_size: int = 1, n_classes: int = 5, output_dimension: int = 512) -> None:
        """
        Initialize Decoder.

        Args:
            pool_output_size: the size of the result feature map after adaptive pooling layer
            n_classes: n classes to output
            output_dimension: output dimension of encoder
        """
        super().__init__()
        #self.pool = nn.AdaptiveAvgPool2d(output_size=pool_output_size)
        self.fc = nn.Linear(output_dimension , n_classes)

    def forward(self, x):
        #x = self.pool(x)
        output = self.fc(x.view(x.size()[0], -1))
        return output

In [11]:
class BasicDecoder_newhead(nn.Module):
    def __init__(self, pool_output_size: int = 1, n_classes: int = 5, output_dimension: int = 512) -> None:
        """
        Initialize Decoder.

        Args:
            pool_output_size: the size of the result feature map after adaptive pooling layer
            n_classes: n classes to output
            output_dimension: output dimension of encoder
        """
        super().__init__()
        #self.pool = nn.AdaptiveAvgPool2d(output_size=pool_output_size)
        self.fc = nn.Sequential(
            nn.Linear(output_dimension, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1),
            nn.Linear(512, 5),
        )

    def forward(self, x):
        #x = self.pool(x)
        output = self.fc(x.view(x.size()[0], -1))
        return output


In [12]:
class Net_B4_newhead(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='tf_efficientnet_b4_ns',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder_newhead(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [13]:
class Net_B4(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='tf_efficientnet_b4_ns',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [14]:
class Net_B5(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='tf_efficientnet_b5_ns',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [15]:
class BasicDecoder_seresnext(nn.Module):
    def __init__(self, pool_output_size: int = 1, n_classes: int = 5, output_dimension: int = 512) -> None:
        """
        Initialize Decoder.

        Args:
            pool_output_size: the size of the result feature map after adaptive pooling layer
            n_classes: n classes to output
            output_dimension: output dimension of encoder
        """
        super().__init__()
        #self.pool = nn.AdaptiveAvgPool2d(output_size=pool_output_size)
        #self.fc = nn.Linear(output_dimension , n_classes)
        self.fc = nn.Sequential(
            nn.Linear(output_dimension, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1),
            nn.Linear(512, 5),
        )
    def forward(self, x):
        #x = self.pool(x)
        output = self.fc(x.view(x.size()[0], -1))
        return output

In [16]:
class seResnextNet(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='seresnext50_32x4d',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder_seresnext(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [17]:
class ResnextNet(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='resnext50_32x4d',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [18]:
class LitCassava(pl.LightningModule):
    def __init__(self, model):
        super(LitCassava, self).__init__()
        self.model = model
        self.metric = pl.metrics.Accuracy()
        self.learning_rate = 1e-4

    def forward(self, x, targets, *args, **kwargs):
        return self.model(x, targets)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.learning_rate, weight_decay=0.001)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=2)

        return (
            [optimizer],
            [{'scheduler': scheduler, 'interval': 'epoch', 'monitor': 'valid_loss'}],
        )

    def training_step(
        self, batch: torch.Tensor, batch_idx: int
    ) -> Union[int, Dict[str, Union[torch.Tensor, Dict[str, torch.Tensor]]]]:
        image = batch['image']
        target = batch['target']
        logits, loss = self(image, target)
        score = self.metric(logits.argmax(1), target)
        logs = {'train_loss': loss, f'train_accuracy': score}
        return {
            'loss': loss,
            'log': logs,
            'progress_bar': logs,
            'logits': logits,
            'target': target,
            f'train_accuracy': score,
        }

    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        y_true = torch.cat([x['target'] for x in outputs])
        y_pred = torch.cat([x['logits'] for x in outputs])
        score = self.metric(y_pred.argmax(1), y_true)
        
        logs = {'train_loss': avg_loss, 'train_accuracy': score}
        return {'log': logs, 'progress_bar': logs}

    def validation_step(
        self, batch: torch.Tensor, batch_idx: int
    ) -> Union[int, Dict[str, Union[torch.Tensor, Dict[str, torch.Tensor]]]]:
        image = batch['image']
        target = batch['target']
        logits, loss = self(image, target)
        score = self.metric(logits.argmax(1), target)
        logs = {'valid_loss': loss, f'valid_accuracy': score}

        return {
            'loss': loss,
            'log': logs,
            'progress_bar': logs,
            'logits': logits,
            'target': target,
            f'valid_accuracy': score,
        }

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        y_true = torch.cat([x['target'] for x in outputs])
        y_pred = torch.cat([x['logits'] for x in outputs])
        score = self.metric(y_pred.argmax(1), y_true)

        # score = torch.tensor(1.0, device=self.device)
        logs = {'valid_loss': avg_loss, f'valid_accuracy': score, 'accuracy': score}
        return {'valid_loss': avg_loss, 'log': logs, 'progress_bar': logs}

In [19]:
labels.head()

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3


In [20]:
# path = "../input/cassava-leaf-disease-classification/"
# sub = pd.read_csv(f'{path}/sample_submission.csv')
# sub.head()

In [21]:
def getloader(df,path,bs=8,fold=0,tta=0):
    
    if tta == 0:
        augs = TTA0
    if tta == 1:
        augs = TTA1
    if tta == 2:
        augs = TTA2
    if tta == 3:
        augs = TTA2_5
    if tta == 4:
        augs = TTA3
    if tta == 5:
        augs = TTA4
    if tta == 6:
        augs = TTA5
    if tta == 7:
        augs = TTA6
    if tta == 8:
        augs = TTA7

    print(f"Using TTA: {tta}")
    folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    train_indexes, valid_indexes = list(folds.split(df, df['label']))[fold]

    train_df = df.iloc[train_indexes]
    valid_df = df.iloc[valid_indexes]


    train_dataset = ImageClassificationDataset(image_names=train_df['image_id'].values,
                                                    transforms=augs,
                                                    labels=train_df['label'].values,
                                                    img_path=path,
                                                    mode='train',
                                                    labels_to_ohe=False,
                                                    n_classes=5)
    valid_dataset = ImageClassificationDataset(image_names=valid_df['image_id'].values,
                                                    transforms=augs,
                                                    labels=valid_df['label'].values,
                                                    img_path=path,
                                                    mode='valid',
                                                    labels_to_ohe=False,
                                                    n_classes=5)
    
    train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=bs,num_workers=0,shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid_dataset,batch_size=bs,num_workers=0,shuffle=False)

    return train_loader,valid_loader,valid_indexes

In [22]:
# models = os.listdir('../input/leaves-tf-enet-b4-ns-576-finetune-temperedloss/')
# modelsPath = [os.path.join('../input/leaves-tf-enet-b4-ns-576-finetune-temperedloss',i) for i in models]
# modelsPath

In [23]:
from tqdm import tqdm
def getPreds(test_loader,lit_model,logitsOut=True):
    predictions = []
    
    for batch in tqdm(test_loader):
        image = batch['image'].to('cuda')
        target = batch['target'].to('cuda')
        with torch.no_grad():
            outputs = lit_model.model(image, target)[0]
            _outputs = outputs.softmax(1).detach().cpu().numpy()
            preds = outputs.argmax(1).detach().cpu().numpy()
            if not logitsOut:
                predictions.append(preds)
            else:
                #predictions[range(len(_outputs))] += _outputs
                predictions.append(_outputs)
    return np.array(predictions)

### Preds for seresnext

In [24]:
#modelsDir = os.listdir(r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\'')

modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\0\epoch=11_valid_loss=0.1303_valid_accuracy=0.8974.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\1\epoch=12_valid_loss=0.1256_valid_accuracy=0.9012.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\2\epoch=11_valid_loss=0.1299_valid_accuracy=0.8997.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\3\epoch=15_valid_loss=0.1374_valid_accuracy=0.8890.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\4\epoch=8_valid_loss=0.1392_valid_accuracy=0.8862.ckpt'}
             


In [25]:
# modelsDir = os.listdir(r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\)

# modelsPath = [os.path.join(r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\best_ones',i) for i in models]
# modelsPath

In [26]:
TTA = 4

In [27]:
preds_seresnext = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
for f in range(5):
    
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = seResnextNet()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
    lit_model.load_state_dict(checkpoint['state_dict'])  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        _,test_loader,valid_ix = getloader(labels,path=train_img_path,bs=128,fold=f,tta=tta)
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_seresnext[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


  0%|          | 0/34 [00:00<?, ?it/s]

Using TTA: 0


100%|██████████| 34/34 [01:41<00:00,  2.99s/it]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/34 [00:00<?, ?it/s]

Using TTA: 1


...

100%|██████████| 34/34 [01:54<00:00,  3.37s/it]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/34 [00:00<?, ?it/s]

Using TTA: 2


100%|██████████| 34/34 [01:54<00:00,  3.37s/it]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/34 [00:00<?, ?it/s]

Using TTA: 3


100%|██████████| 34/34 [01:54<00:00,  3.38s/it]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [28]:
preds_mean_seresnext = preds_seresnext.copy()
preds_mean_seresnext[:,0]= preds_mean_seresnext[:,0]/(1+np.sqrt(0.05))
preds_mean_seresnext[:,1]= preds_mean_seresnext[:,1]/(1+np.sqrt(0.102))
preds_mean_seresnext[:,2]= preds_mean_seresnext[:,2]/(1+np.sqrt(0.111))
preds_mean_seresnext[:,3]= preds_mean_seresnext[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_seresnext[:,4]= preds_mean_seresnext[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_seresnext,1)

In [29]:
# classOut = np.argmax(preds_seresnext,1)
# classOut.shape

In [30]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_seresnext,1))

(0.8962471374491752, 0.8967144926858905)

In [31]:
del _,softmaxOut,model,lit_model,checkpoint,test_loader; gc.collect()

300

In [32]:
torch.cuda.empty_cache()

In [33]:
gc.collect()

40

### Preds for B4 focal cosine loss

In [34]:
modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\0\epoch=21_valid_loss=0.1313_valid_accuracy=0.8988.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\1\epoch=11_valid_loss=0.1244_valid_accuracy=0.9070.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\2\epoch=7_valid_loss=0.1304_valid_accuracy=0.8967.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\3\epoch=17_valid_loss=0.1344_valid_accuracy=0.8913.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\4\epoch=13_valid_loss=0.1341_valid_accuracy=0.8946.ckpt'
             }


In [35]:
# modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\0\epoch=4_valid_loss=0.0972_valid_accuracy=0.8949.ckpt',
#               1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\1\epoch=4_valid_loss=0.0919_valid_accuracy=0.9012.ckpt',
#               2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\2\epoch=8_valid_loss=0.0967_valid_accuracy=0.8923.ckpt',
#               3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\3\epoch=8_valid_loss=0.1017_valid_accuracy=0.8885.ckpt',
#               4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\4\epoch=0_valid_loss=0.1005_valid_accuracy=0.8913.ckpt'
#              }
             


In [36]:
preds_effnet_b4 = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = TTA
for f in range(5):    
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = Net_B4_newhead()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
    lit_model.load_state_dict(checkpoint['state_dict'])  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        _,test_loader,valid_ix = getloader(labels,path=train_img_path,bs=32,fold=f,tta=tta)
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_effnet_b4[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 0


100%|██████████| 134/134 [01:32<00:00,  1.45it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1


...

100%|██████████| 134/134 [01:22<00:00,  1.63it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 2


100%|██████████| 134/134 [01:22<00:00,  1.62it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 3


100%|██████████| 134/134 [01:22<00:00,  1.63it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [37]:
preds_mean_effnetb4 = preds_effnet_b4.copy()
preds_mean_effnetb4[:,0]= preds_effnet_b4[:,0]/(1+np.sqrt(0.05))
preds_mean_effnetb4[:,1]= preds_effnet_b4[:,1]/(1+np.sqrt(0.102))
preds_mean_effnetb4[:,2]= preds_effnet_b4[:,2]/(1+np.sqrt(0.111))
preds_mean_effnetb4[:,3]= preds_effnet_b4[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_effnetb4[:,4]= preds_effnet_b4[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_effnetb4,1)

In [38]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_effnet_b4,1))

(0.8982567649670514, 0.8988643267747816)

In [39]:
del _,softmaxOut,model,lit_model,checkpoint,test_loader; gc.collect();torch.cuda.empty_cache()

### Ensemble

In [51]:
res = pd.DataFrame()
res['b4_contrib'] = 0
#res['b5_contrib'] = 0
res['seresnext_contrib'] = 0
res ['Accuracy'] = 0

count = 0
for i in np.arange(0,1,0.01):
    b4_contrib = i
    seresnext_contrib = 1-i
    #print(i,j,1-i-j)
    adj_preds = (b4_contrib*preds_mean_effnetb4+seresnext_contrib*preds_mean_seresnext)
    ens_preds = (b4_contrib*preds_effnet_b4+seresnext_contrib*preds_seresnext)
    res.loc[count,'b4_contrib'] = i
    res.loc[count,'seresnext_contrib'] = 1-i
    res.loc[count,'Adj_Accuracy'] = accuracy_score(labels.label,np.argmax(adj_preds,1))
    res.loc[count,'Naive_Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
    count += 1

In [52]:
res.sort_values(by=['Adj_Accuracy'],ascending=False)

,b4_contrib,seresnext_contrib,Accuracy,Adj_Accuracy,Naive_Accuracy
18,0.18,0.82,NaN,0.900640,0.900547
19,0.19,0.81,NaN,0.900594,0.900687
17,0.17,0.83,NaN,0.900500,0.900360
20,0.20,0.80,NaN,0.900453,0.900780
24,0.24,0.76,NaN,0.900173,0.901014
...,...,...,...,...,...
4,0.04,0.96,NaN,0.897556,0.898818
3,0.03,0.97,NaN,0.896901,0.898490
2,0.02,0.98,NaN,0.896761,0.898070
1,0.01,0.99,NaN,0.896294,0.897462


In [53]:
res['Naive_Accuracy'].max(),res['Adj_Accuracy'].max()

(0.9011543674346871, 0.9006402766743001)

### Preds for B5

In [42]:
modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\0\epoch=0_valid_loss=0.0980_valid_accuracy=0.8949.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\1\epoch=4_valid_loss=0.0926_valid_accuracy=0.9021.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\2\epoch=7_valid_loss=0.0942_valid_accuracy=0.8986.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\3\epoch=4_valid_loss=0.1031_valid_accuracy=0.8883.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\4\epoch=3_valid_loss=0.1046_valid_accuracy=0.8890.ckpt'
             }
             


In [43]:
preds_effnet_b5 = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = TTA
for f in range(5):
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = Net_B5()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
    lit_model.load_state_dict(checkpoint['state_dict'])  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        _,test_loader,valid_ix = getloader(labels,path=train_img_path,bs=32,fold=f,tta=tta)
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_effnet_b5[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 0


100%|██████████| 134/134 [01:33<00:00,  1.43it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1


...

100%|██████████| 134/134 [01:33<00:00,  1.43it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 2


100%|██████████| 134/134 [01:34<00:00,  1.42it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 3


100%|██████████| 134/134 [01:34<00:00,  1.42it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [44]:
preds_mean_effnetb5 = preds_effnet_b5.copy()
preds_mean_effnetb5[:,0]= preds_effnet_b5[:,0]/(1+np.sqrt(0.05))
preds_mean_effnetb5[:,1]= preds_effnet_b5[:,1]/(1+np.sqrt(0.102))
preds_mean_effnetb5[:,2]= preds_effnet_b5[:,2]/(1+np.sqrt(0.111))
preds_mean_effnetb5[:,3]= preds_effnet_b5[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_effnetb5[:,4]= preds_effnet_b5[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_effnetb5,1)

In [45]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_effnet_b5,1))

(0.896667757162219, 0.8970883768752629)

In [46]:
del _,softmaxOut,model,lit_model,checkpoint,test_loader; gc.collect();torch.cuda.empty_cache()

### Ensemble

In [47]:
res = pd.DataFrame()
res['b4_contrib'] = 0
res['b5_contrib'] = 0
res['seresnext_contrib'] = 0
res ['Accuracy'] = 0

count = 0
for i in np.arange(0,1,0.01):
    for j in np.arange(0,1-i,0.01):
        b4_contrib = i
        b5_contrib = j
        seresnext_contrib = 1-i-j
        #print(i,j,1-i-j)
        adj_preds = (b4_contrib*preds_mean_effnetb4+seresnext_contrib*preds_mean_seresnext+b5_contrib*preds_mean_effnetb5)
        ens_preds = (b4_contrib*preds_effnet_b4+seresnext_contrib*preds_seresnext+b5_contrib*preds_effnet_b5)
        res.loc[count,'b4_contrib'] = i
        res.loc[count,'seresnext_contrib'] = 1-i-j
        res.loc[count,'b5_contrib'] = j
        #res.loc[count,'Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
        res.loc[count,'Adj_Accuracy'] = accuracy_score(labels.label,np.argmax(adj_preds,1))
        res.loc[count,'Naive_Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
        count += 1

In [48]:
res.sort_values(by=['Adj_Accuracy'],ascending=False)

,b4_contrib,b5_contrib,seresnext_contrib,Accuracy,Adj_Accuracy,Naive_Accuracy
1676,0.18,0.29,0.53,NaN,0.902650,0.903304
1842,0.20,0.32,0.48,NaN,0.902603,0.903117
1614,0.17,0.50,0.33,NaN,0.902603,0.901809
1519,0.16,0.39,0.45,NaN,0.902603,0.902510
1771,0.19,0.42,0.39,NaN,0.902556,0.902089
...,...,...,...,...,...,...
199,0.02,0.00,0.98,NaN,0.896761,0.898070
2,0.00,0.02,0.98,NaN,0.896714,0.898117
101,0.01,0.01,0.98,NaN,0.896668,0.898444
100,0.01,0.00,0.99,NaN,0.896294,0.897462


In [50]:
res['Naive_Accuracy'].max(),res['Adj_Accuracy'].max()

(0.9034444080945927, 0.9026499041921765)

### Effnet B6 - TBD

In [ ]:
modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\0\epoch=0_valid_loss=0.0980_valid_accuracy=0.8949.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\1\epoch=4_valid_loss=0.0926_valid_accuracy=0.9021.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\2\epoch=7_valid_loss=0.0942_valid_accuracy=0.8986.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\3\epoch=4_valid_loss=0.1031_valid_accuracy=0.8883.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\4\epoch=3_valid_loss=0.1046_valid_accuracy=0.8890.ckpt'
             }
             


### Ensemble

In [ ]:
i = 0.4
np.arange(0,1-i,0.01)

In [ ]:
res = pd.DataFrame()
res['b4_contrib'] = 0
res['b5_contrib'] = 0
res['seresnext_contrib'] = 0
res ['Accuracy'] = 0

count = 0
for i in np.arange(0,1,0.01):
    for j in np.arange(0,1-i,0.01):
        b4_contrib = i
        b5_contrib = j
        seresnext_contrib = 1-i-j
        #print(i,j,1-i-j)
        ens_preds = (b4_contrib*preds_effnet_b4+seresnext_contrib*preds_seresnext+b5_contrib*preds_effnet_b5)
        res.loc[count,'b4_contrib'] = i
        res.loc[count,'seresnext_contrib'] = 1-i-j
        res.loc[count,'b5_contrib'] = j
        res.loc[count,'Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
        count += 1

In [ ]:
res.sort_values(by=['Accuracy'],ascending=False)